In [20]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [21]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

NotImplementedError: Cannot convert a symbolic Tensor (lstm_11/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [4]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ['acc'])

history = model.fit(train_data, train_labels, epochs = 10, validation_split = 0.2)

Epoch 1/10
625/625 [==============================] - 32s 50ms/step - loss: 0.4176 - acc: 0.8138 - val_loss: 0.3017 - val_acc: 0.8830
Epoch 2/10
625/625 [==============================] - 32s 52ms/step - loss: 0.2379 - acc: 0.9087 - val_loss: 0.3112 - val_acc: 0.8794
Epoch 3/10
625/625 [==============================] - 33s 52ms/step - loss: 0.1843 - acc: 0.9327 - val_loss: 0.2828 - val_acc: 0.8782
Epoch 4/10
625/625 [==============================] - 33s 53ms/step - loss: 0.1527 - acc: 0.9456 - val_loss: 0.2802 - val_acc: 0.8994
Epoch 5/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1309 - acc: 0.9542 - val_loss: 0.2938 - val_acc: 0.8884
Epoch 6/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1108 - acc: 0.9624 - val_loss: 0.5050 - val_acc: 0.8608
Epoch 7/10
625/625 [==============================] - 34s 55ms/step - loss: 0.0967 - acc: 0.9678 - val_loss: 0.3270 - val_acc: 0.8848
Epoch 8/10
625/625 [==============================] - 34s 55ms

In [ ]:
res = model.evaluate(test_data, test_labels)

print(res)

782/782 [==============================] - 11s 15ms/step - loss: 0.4329 - acc: 0.8554
[0.43285879492759705, 0.8553599715232849]


In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [ ]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.8664749]
[0.4365361]


In [4]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [5]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
seq_length = 100 
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  
    input_text = chunk[:-1] 
    target_text = chunk[1:]  
    return input_text, target_text  

dataset = sequences.map(split_input_target)  

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  
EMBEDDING_DIM = 256
RNN_UNITS = 1024
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [12]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.85384881e-03 -7.15153897e-03  1.15660101e-03 ... -1.92732131e-03
    1.01137888e-02  1.42219325e-03]
  [-5.53619768e-03 -1.00687137e-02  1.20080775e-03 ...  4.36535105e-03
    5.02312602e-03 -9.55781434e-05]
  [-3.62341525e-03 -9.68053006e-03 -2.61474925e-04 ...  6.40594028e-03
    5.06449817e-03  3.20428354e-03]
  ...
  [-4.62308293e-03 -9.21891909e-03 -3.64783593e-03 ... -1.83762447e-03
    8.26505106e-03  7.79851526e-03]
  [-7.68918172e-03 -1.02117928e-02 -6.65181642e-03 ... -5.93991950e-03
    1.09887589e-03  4.06720676e-03]
  [-6.90427097e-03 -4.28111851e-03 -5.35473134e-03 ... -4.33838274e-03
   -1.11915823e-03  3.62640619e-03]]

 [[-4.29799780e-03 -4.09796182e-03 -3.31965042e-03 ... -3.55226779e-03
   -4.97486535e-03 -8.35439132e-05]
  [-4.50156815e-03 -4.77675349e-04 -2.32855696e-03 ... -1.81980967e-03
   -5.68757951e-03  1.70415395e-03]
  [-2.15397216e-03 -8.97433981e-03 -1.30563625e-03 ... -3.85764427e-03
    5.24592213e-03  3.36068962e-03]
  ...
  [-3.951

In [13]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)


100
tf.Tensor(
[[ 1.8538488e-03 -7.1515390e-03  1.1566010e-03 ... -1.9273213e-03
   1.0113789e-02  1.4221933e-03]
 [-5.5361977e-03 -1.0068714e-02  1.2008077e-03 ...  4.3653511e-03
   5.0231260e-03 -9.5578143e-05]
 [-3.6234153e-03 -9.6805301e-03 -2.6147492e-04 ...  6.4059403e-03
   5.0644982e-03  3.2042835e-03]
 ...
 [-4.6230829e-03 -9.2189191e-03 -3.6478359e-03 ... -1.8376245e-03
   8.2650511e-03  7.7985153e-03]
 [-7.6891817e-03 -1.0211793e-02 -6.6518164e-03 ... -5.9399195e-03
   1.0988759e-03  4.0672068e-03]
 [-6.9042710e-03 -4.2811185e-03 -5.3547313e-03 ... -4.3383827e-03
  -1.1191582e-03  3.6264062e-03]], shape=(100, 65), dtype=float32)


In [14]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)


65
tf.Tensor(
[ 1.85384881e-03 -7.15153897e-03  1.15660101e-03  1.73294707e-03
 -1.63763855e-03 -1.38659007e-03  1.18519575e-03  5.56458626e-03
  5.54338680e-04  3.49276327e-03  9.96226212e-04 -3.80037772e-03
 -4.40398837e-03 -5.04954718e-04 -4.70775925e-03  6.96346350e-03
  7.42519973e-04 -2.88586598e-05  1.52442697e-03  1.25835557e-03
 -7.22266501e-03 -3.40661500e-04  2.57383566e-04  3.44857201e-03
  1.63050671e-03 -4.52300161e-03  1.08457636e-04  2.33771093e-03
  5.62603120e-03 -3.96918040e-04  7.10480846e-04 -9.82050318e-04
 -1.16226531e-03 -1.30103761e-03  4.02910775e-03 -4.35928523e-04
  1.84874982e-03 -5.55540901e-05 -2.80846492e-03 -1.92014966e-04
  1.40938128e-03 -5.20899659e-03 -1.19606964e-03 -3.89274932e-03
 -2.99509522e-03  3.19387671e-03 -2.28387862e-03  7.24914134e-04
  4.75809257e-03  1.08334341e-03 -1.34408148e-03  1.00957055e-04
  6.46665413e-03 -8.36011302e-03 -4.27494477e-03  2.38120835e-03
 -2.86889146e-03 -6.40788407e-04 -6.56149350e-05  5.39351488e-03
  2.8600150

In [15]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [16]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars 

"rXnzPDgn&GP,'eIWx!?eRP:ORUcMyJVmZ:sU'gH VIBj.Q$q3Xm3?ikfGgYdzAl?PuF3oYtMgcIE bopFG,biZ.Gs:FYyDrf.:gK"

In [17]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [18]:
model.compile(optimizer='adam', loss=loss)

In [19]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 19s 98ms/step - loss: 2.6105
Epoch 2/50
172/172 [==============================] - 18s 99ms/step - loss: 1.8981
Epoch 3/50
172/172 [==============================] - 18s 100ms/step - loss: 1.6474
Epoch 4/50
172/172 [==============================] - 18s 100ms/step - loss: 1.5111
Epoch 5/50
172/172 [==============================] - 18s 100ms/step - loss: 1.4293
Epoch 6/50
172/172 [==============================] - 18s 101ms/step - loss: 1.3726
Epoch 7/50
172/172 [==============================] - 18s 101ms/step - loss: 1.3264
Epoch 8/50
172/172 [==============================] - 18s 101ms/step - loss: 1.2874
Epoch 9/50
172/172 [==============================] - 18s 101ms/step - loss: 1.2515
Epoch 10/50
172/172 [==============================] - 18s 101ms/step - loss: 1.2176
Epoch 11/50
172/172 [==============================] - 18s 102ms/step - loss: 1.1828
Epoch 12/50
172/172 [==============================] - 18s 102ms/step - loss

In [21]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [22]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [25]:
def generate_text(model, start_string):
  num_generate = 800

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []


  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
    
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Romeou of my intelligence, man?
Here he comeslander with a fool?
Upon the heaven, that curts it tainted ere use these services
My warding a musician: therefore I
Do not believe them in the angry doy,
That thoughts before the fire of such rebellion
That would unhappy days befall you born,
And I'll camb him here she must be hold;
I hope you will not prove a troublous world.

AUTOLYCUS:

Clown:
My lord, for that which you promise me, and I'll use to adish'd him
When he hath toached him by any but love;
Who dares surning
That thought them thence is just, proclaim a several trul.

BAPTISTA:
The devil than Angelo; I cannot do it;
That if thou never verke here of Banish here;
And any thing, with nothing officer
Prepare her earth; the wors abe now, that makes me bad: Here comes a man,
Daughter, were my
